Code to get $chi^2$ results to Claudio

Created by Linnea on October 11, 2023

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf
from pathlib import Path
from collections import defaultdict
import h5py
import keras_core as keras
import tensorflow_io as tfio

import sys
sys.path.append('../')
import preprocess.preprocess
import utils
import jax
import jax.numpy as jnp

%load_ext autoreload
%autoreload 2

2023-10-31 19:45:39.110683: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-31 19:45:40.097869: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-31 19:45:40.118715: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-31 19:47:15.207928: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Using TensorFlow backend


In [2]:
# Load metadata for this interval
def index_mcmc_runs():
    """Make a list of combinations for which we want to run MCMC."""
    experiments = ['AMS02_H-PRL2021']
    dfs = []
    for experiment_name in experiments:
        filename = f'../../data/2023/{experiment_name}_heliosphere.dat'
        df = utils.index_experiment_files(filename) 
        df['experiment_name'] = experiment_name
        df['filename_heliosphere'] = filename
        dfs.append(df)
    df = pd.concat(dfs, axis=0, ignore_index=0)
    return df

# Select experiment parameters
df = index_mcmc_runs()  # List of all ~200 experiments.
data = df.iloc[0]

In [3]:
# Load parameters
polarity = 'neg'
path = '/home/linneamw/sadow_koastore/personal/linneamw/research/gcr/data/2023_07_01'
f_original = f'{path}/{polarity}/model_collection_1AU_90deg_0deg.h5'

# 8 input parameters for the NN: alpha, cmf, vspoles, cpa, pwr1par, pwr2par, pwr1perr, and pwr2perr.
# features = ['alpha', 'cmf', 'cpa', 'pwr1par', 'pwr1perr', 'pwr2par', 'pwr2perr', 'vspoles']
with h5py.File(f_original, 'r') as h5:
    print(h5.keys())

    info = h5['info']
    model = h5['model']

    print(info.keys())
    print(model.keys())

    # Get relevant parameters
    alpha = h5['model/alpha'][:]
    cmf = h5['model/cmf'][:]
    cpa = h5['model/cpa'][:]
    pwr1par = h5['model/pwr1par'][:]
    pwr1perr = h5['model/pwr1perr'][:]
    pwr2par = h5['model/pwr2par'][:]
    pwr2perr = h5['model/pwr2perr'][:]
    vspoles = h5['model/vspoles'][:]
    imodel = h5['model/imodel'][:]
    rigidity = h5['info/rigidity'][:]

<KeysViewHDF5 ['info', 'model']>
<KeysViewHDF5 ['LIS', 'rigidity']>
<KeysViewHDF5 ['alpha', 'cmf', 'cpa', 'flux', 'imodel', 'ipar', 'pwr1par', 'pwr1perr', 'pwr2par', 'pwr2perr', 'quality', 'vseq', 'vspoles']>


In [4]:
# Find parameters associated with index 1889802
index = 1889802

# Load parameters for NN: ['alpha', 'cmf', 'cpa', 'pwr1par', 'pwr1perr', 'pwr2par', 'pwr2perr', 'vspoles']
parameters = (alpha[index], cmf[index], cpa[index], pwr1par[index], pwr1perr[index], pwr2par[index], pwr2perr[index], vspoles[index])

print(f'imodel: {imodel[index]}')
print(f'(alpha, cmf, cpa, pwr1par, pwr1perr, pwr2par, pwr2perr, vspoles): {parameters}')

imodel: 1889802
(alpha, cmf, cpa, pwr1par, pwr1perr, pwr2par, pwr2perr, vspoles): (55.0, 4.5, 390.0, 1.3, 1.0, 2.3, 0.4, 600.0)


In [17]:
model_version = 'v3.0'
data_path = f'../../data/oct2022/{data.experiment_name}/{data.experiment_name}_{data.interval}.dat'  # This data is the same.
model_path = f'../../models/model_{model_version}_{data.polarity}.keras'
penalty = 1e6

print(f'model_path: {model_path}')
print(f'data_path: {data_path}')

# Load trained NN model that maps 7 parameters to predicted flux at RIGIDITY_VALS.
model = keras.models.load_model(model_path)
model.run_eagerly = True # Settable attribute (in elegy). Required to be true for ppmodel.

# Load observation data from Claudio
bins, observed, uncertainty = utils.load_data_ams(data_path)
bin_midpoints = (bins[:-1] * bins[1:]) ** 0.5  # Geometric mean seemed to work better in exp.

# Transform parameters
parameters_transformed = utils.transform_input(jnp.array(parameters))

# Predict flux
xs = parameters_transformed.reshape((1,-1)) # Reshape to (,8) for keras.
yhat = model(xs)
yhat = yhat.numpy()[0] # Convert to float32
yhat = utils.untransform_output(yhat.reshape((1,-1))).reshape(-1) # Undo scaling and minmax.
log_yhat = np.log(yhat)

model_path: ../../models/model_v3.0_neg.keras
data_path: ../../data/oct2022/AMS02_H-PRL2021/AMS02_H-PRL2021_20110520-20110610.dat


## Way 1: interpolated chi^2, log-log interpolation at geometric mean of midpoint

In [18]:
# Interpolate to get predicted flux at midpoint of bin points.
yhat_interp = jnp.interp(bin_midpoints, utils.RIGIDITY_VALS, log_yhat)

# Compute log prob
chi2_interp = (((yhat_interp - observed)/uncertainty)**2)
residual_interp = (yhat_interp - observed)/uncertainty

print(f'chi2_interp: {chi2_interp}')

chi2_interp: [1.1355219e+03 1.8906440e+03 2.8022266e+03 3.7665627e+03 4.6596187e+03
 5.4470601e+03 6.0993247e+03 6.6741899e+03 7.1358501e+03 7.5271826e+03
 7.7285498e+03 7.8565991e+03 7.8701948e+03 7.9878838e+03 7.9579014e+03
 7.7289805e+03 7.4963135e+03 7.2905845e+03 7.0844023e+03 6.7085312e+03
 6.3673843e+03 6.1410981e+03 5.7708857e+03 5.2328916e+03 4.2016177e+03
 3.0766404e+03 3.2941108e+03 2.4305336e+04 4.6356000e+05 7.0275215e+06]


## Way 2: integrated chi^2, using Claudio's code to calculate the integral

In [19]:
class ModelChi2:
    def __init__(self, mLogR, mLogF):
        self.mLogR = mLogR
        self.mLogF = mLogF

    def interpolate_model(self, x):
        lnx = np.log(x)
        ig = self.find_grid_point(lnx)

        print(f"Interpolating model for x: {x:.3f}, which has self.mLogR[ig]: {self.mLogR[ig]:.3f} and self.mLogR[ig+1]: {self.mLogR[ig+1]:.3f}, and self.mLogF[ig]: {self.mLogF[ig]:.3f} and self.mLogF[ig+1]: {self.mLogF[ig+1]:.3f}")

        s = (lnx - self.mLogR[ig]) / (self.mLogR[ig+1] - self.mLogR[ig])
        result = np.exp((1 - s) * self.mLogF[ig] + s * self.mLogF[ig+1])

        return result

    def compute_integral(self, x1, x2):
        lnx1 = np.log(x1)
        lnx2 = np.log(x2)
        ig1 = self.find_grid_point(lnx1)
        ig2 = self.find_grid_point(lnx2)

        # Compute integral from a to b at grid point ig
        def integral(a, b, ig):
            assert ig < len(self.mLogR), f"ArithmeticError: ig:{ig}> len(self.mLogR):{len(self.mLogR)}"
            assert ig < len(self.mLogF), f"ArithmeticError: ig:{ig}> len(self.mLogF):{len(self.mLogF)}"

            print(f"Computing integral from {a:.3f} to {b:.3f}, which has self.mLogR[ig]: {self.mLogR[ig]:.3f} and self.mLogR[ig+1]: {self.mLogR[ig+1]:.3f}, and self.mLogF[ig]: {self.mLogF[ig]:.3f} and self.mLogF[ig+1]: {self.mLogF[ig+1]:.3f}")

            M = (self.mLogF[ig+1] - self.mLogF[ig]) / (self.mLogR[ig+1] - self.mLogR[ig])
            N = np.exp(self.mLogF[ig] - M * self.mLogR[ig])
            Mp1 = M + 1
            N_mp1 = N / Mp1
            mp1b_mp1a = np.exp(Mp1 * b) - np.exp(Mp1 * a)
            result = N_mp1 * mp1b_mp1a

            print(f'M: {M:.3f}, N: {N:.3f}, Mp1: {Mp1:.3f}, N_mp1: {N_mp1:.3f}, mp1b_mp1a: {mp1b_mp1a:.3f}')
            print(f'Computed integral: {result:.3f}')

            return result

        # Compute integral from lnx1 to min(lnx2, self.mLogR[ig1+1]) at grid point ig1
        assert ig1+1 < len(self.mLogR), f"ArithmeticError: ig1+1:{ig1+1}> len(self.mLogR):{len(self.mLogR)}"
        I = integral(lnx1, min(lnx2, self.mLogR[ig1+1]), ig1)

        # For each grid point between ig1+1 and ig2, compute integral from R[ig] to min(lnx2, R[ig+1]) at grid point ig
        for ig in range(ig1+1, ig2+1):
            assert ig+1 < len(self.mLogR), f"ArithmeticError: ig+1:{ig+1}> len(self.mLogR):{len(self.mLogR)}"
            I += integral(self.mLogR[ig], min(lnx2, self.mLogR[ig+1]), ig)
        
        # If lnx2 > R[ig2+1], compute integral from R[ig2+1] to lnx2 at grid point ig2
        if lnx2 > self.mLogR[ig2+1]:
            assert ig2+1 < len(self.mLogR), f"ArithmeticError: ig2+1:{ig2+1}> len(self.mLogR):{len(self.mLogR)}"
            I += integral(self.mLogR[ig2+1], lnx2, ig2)

        return I

    # find grid point ig such that R[ig] <= x <= R[ig+1]
    def find_grid_point(self, value): # Checked: this works correctly. 
        result = np.searchsorted(self.mLogR, value, side="left")-1
        
        if result >= len(self.mLogR)-1:
            result = len(self.mLogR)-2

        print(f'Finding grid_point for the value: {value:.3f}, self.mLogR[result]: {self.mLogR[result]:.3f} and self.mLogR[result+1]: {self.mLogR[result+1]:.3f}')

        return result

# Usage:
# mLogR = your mLogR data as a numpy array, in logspace
# mLogF = your mLogF data as a numpy array, in logspace
# model = ModelChi2(mLogR, mLogF)
# result = model.compute_integral(x1, x2)

In [20]:
# Get log_rigidity and log_yhat
log_rigidity = np.log(utils.RIGIDITY_VALS)

print(f'bins: {len(bins)}\n{bins}\n\n')
print(f'log_yhat: {len(log_yhat)}\n{log_yhat}')
print(f'log_rigidity: {len(log_rigidity)}\n{log_rigidity}')

# Initialize model
model = ModelChi2(log_rigidity, log_yhat)
yhat_integral = []
yhat_interp = []

# Compute integral for each bin
for x1, x2 in zip(bins[:-1], bins[1:]):
    print(f'\nCurrent x1: {x1}, x2: {x2}')
    integral = model.compute_integral(x1, x2) / (x2 - x1)

    print(f'Integral: {integral:.3f}')
    yhat_integral.append(integral)

# Compute interpolated value for each bin
for x in bin_midpoints:
    print(f'\nx: {x}')
    interpolated = model.interpolate_model(x)

    print(f'Interpolated: {interpolated:.3f}')
    yhat_interp.append(interpolated)

print(f'yhat_integral: {len(yhat_integral)}\n{yhat_integral}')
print(f'yhat_interp: {len(yhat_interp)}\n{yhat_interp}')

bins: 31
[  1.     1.16   1.33   1.51   1.71   1.92   2.15   2.4    2.67   2.97
   3.29   3.64   4.02   4.43   4.88   5.37   5.9    6.47   7.09   7.76
   8.48   9.26  10.1   11.    13.    16.6   22.8   33.5   48.5   69.7
 100.  ]


log_yhat: 32
[ 3.589842    3.6240537   3.6784282   3.7757766   3.9370165   4.115487
  4.3290825   4.594681    4.88857     5.189218    5.537966    5.875779
  6.1928697   6.489793    6.7057624   6.8199687   6.814575    6.6836143
  6.423785    6.036938    5.5648704   4.954528    4.268897    3.4731126
  2.568606    1.6393415   0.63703626 -0.40959284 -1.5269916  -2.6721764
 -3.8564389  -5.032763  ]
log_rigidity: 32
[-1.60943791 -1.59862765 -1.57700712 -1.54457635 -1.49052507 -1.42566349
 -1.34999171 -1.25269936 -1.1445968  -1.02568395 -0.88515058 -0.73380696
 -0.57165308 -0.38787868 -0.19329403  0.01210089  0.23911632  0.47694201
  0.72557796  0.99583442  1.26609089  1.56877813  1.87146537  2.19577313
  2.54170141  2.88762969  3.25517848  3.63353753  4.0335171   

In [21]:
# Compute log prob for integral
chi2_integral = (((yhat_integral - observed)/uncertainty)**2)
residual_integral = (yhat_integral - observed)/uncertainty

# Compute log prob for interpolation
chi2_interp = (((yhat_interp - observed)/uncertainty)**2)
residual_interp = (yhat_interp - observed)/uncertainty

print(f'chi2_integral: {chi2_integral}')
print(f'chi2_interp: {chi2_interp}')

chi2_integral: [2.01741124 2.44509114 1.4323885  0.22164475 0.05371504 0.63194258
 0.99959153 2.91828136 6.45423456 6.09950023 9.22666034 5.63385252
 3.90386468 6.86793821 4.24053465 2.58315052 3.22433777 2.59893809
 0.74362674 0.20273395 0.25994414 0.11325549 1.09244452 2.65839684
 5.35112926 8.18279546 9.75248905 8.16145331 5.21564299 4.74068413]
chi2_interp: [1.98893210e+00 1.90756361e+00 1.36628265e+00 9.36789519e-07
 3.74221616e-02 1.07970857e+00 1.06634628e+00 3.03066826e+00
 6.68031351e+00 6.18640974e+00 9.83069947e+00 5.62783494e+00
 3.89906921e+00 7.02659660e+00 4.15139212e+00 2.51668017e+00
 3.15327692e+00 2.48968956e+00 6.84761094e-01 1.74053582e-01
 2.99724554e-01 1.45542669e-01 1.18367987e+00 3.17727627e+00
 7.25868209e+00 1.21640362e+01 1.63821262e+01 1.41818191e+01
 1.03189667e+01 8.97056409e+00]


## Print final solution

In [22]:
# For each data bin, print the rigidity (geometric mean + left and right edges); the data value and uncertainty; 
# the model interpolated in the geometric mean and its residual (data - model)/uncertainty; 
# the integral of the model over the bin and its residual; and the cumulative chi2 using the model integral.

for i in range(len(bin_midpoints)):
    print(f'[0{i}] rig = {bin_midpoints[i]:.6f} [{bins[i]:.2f}, {bins[i+1]:.2f}]; data = {observed[i]:.6e}; unc = {uncertainty[i]:.6e}; mod = {yhat_interp[i]:.6e}; res = {residual_interp[i]:+.6e}; mod_int = {yhat_integral[i]:.6e}; res_int = {residual_integral[i]:+.6e}; cum_chi2_interpolated = {np.sum(chi2_interp[:i+1]):.6e}; cum_chi2_integrated = {np.sum(chi2_integral[:i+1]):.6e}')

[00] rig = 1.077033 [1.00, 1.16]; data = 9.542576e+02; unc = 2.811600e+01; mod = 9.146057e+02; res = -1.410295e+00; mod_int = 9.143229e+02; res_int = -1.420356e+00; cum_chi2_interpolated = 1.988932e+00; cum_chi2_integrated = 2.017411e+00
[01] rig = 1.242095 [1.16, 1.33]; data = 9.411921e+02; unc = 2.148905e+01; mod = 9.115126e+02; res = -1.381146e+00; mod_int = 9.075901e+02; res_int = -1.563679e+00; cum_chi2_interpolated = 3.896496e+00; cum_chi2_integrated = 4.462502e+00
[02] rig = 1.417145 [1.33, 1.51]; data = 8.769211e+02; unc = 1.643800e+01; mod = 8.577070e+02; res = -1.168881e+00; mod_int = 8.572477e+02; res_int = -1.196824e+00; cum_chi2_interpolated = 5.262778e+00; cum_chi2_integrated = 5.894891e+00
[03] rig = 1.606891 [1.51, 1.71]; data = 8.003528e+02; unc = 1.293201e+01; mod = 8.003653e+02; res = +9.678789e-04; mod_int = 7.942645e+02; res_int = -4.707916e-01; cum_chi2_interpolated = 5.262779e+00; cum_chi2_integrated = 6.116536e+00
[04] rig = 1.811960 [1.71, 1.92]; data = 7.08868